In [1]:
from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import google_auth_httplib2
import html
import TracklistGenerator
import YouTubeSearchEngine
import time
import YouTubeDownloader
from FetchMP3 import getmp3
from SpotifyIntegration import SpotifyConnector
# Global Declaration
globalTracklistName = 0
client_secret_file = 'client_secret.json'
SCOPES = ['https://www.googleapis.com/auth/youtube']

def OAuthVerification():
    flow = InstalledAppFlow.from_client_secrets_file(client_secret_file,SCOPES)
    creds = flow.run_console()
    youtube = build('youtube','v3',credentials=creds)
    return youtube 
# Returning Youtube Object

# Insert Playlist
def insertPlaylist(youtube,tracklistCleanedName,cleanedTracks):
    insertplaylist = youtube.playlists().insert(part="snippet,status",body={
              "snippet": {
                "title": tracklistCleanedName,
                "description": tracklistCleanedName
              },
              "status": {
                "privacyStatus": "public"
              }
            })
    details = insertplaylist.execute()
    print(details)
    print(details['id'])
    playlistId = details['id']
    return playlistId

# Search for a Video element
def searchUsingAPI(youtube,trackTitle):
        searchResult = youtube.search().list(part='snippet',q=trackTitle,type='video',maxResults=20).execute()
        videoTitles = {}
        for snippet in searchResult['items']:
            videoId = snippet['id']['videoId']
            vidTitle = snippet['snippet']['title']
            videoTitles[videoId] = html.unescape(vidTitle)
        print(videoTitles)
                
        # -----------Video Compare Logic----------------
        keys = videoTitles.keys()
        matchResults = {}
        for key in keys:
            print(videoTitles[key])
            if str(videoTitles[key]).lower() == html.unescape(trackTitle).lower():
                matchResults[key] = 1
            else:
                matchResults[key] = 0
        print(matchResults)
         
        # -----------------Actual Video Id-----------------
        relativeVideoIds = [] # Which varies based on video title 
        actualVideoIds = [] # List of videos which are exact match with the title 
        matchResultsKeys = matchResults.keys()
        for eachKey in matchResultsKeys:
            if int(matchResults[eachKey]) == 1:
                actualVideoIds.append(eachKey)
            else:
                relativeVideoIds.append(eachKey)
        print("Relative Videos",relativeVideoIds)  
        print("Actual Videos",actualVideoIds)
        
        # -----------------Adding Videos to Playlist------------
        if actualVideoIds:
            actualVideoId = actualVideoIds[0]
            print("Getting Actual Video",actualVideoId)
            return actualVideoId
        elif relativeVideoIds:
            actualVideoId = relativeVideoIds[0]
            print("Getting Relative Video",actualVideoId)
            return actualVideoId
        else:
            return "Video Not Found"
    
def insertTracks(youtube,playlistId,cleanedTracks):
    print("Creating your Youtube Playlist. It may take a while....")
    playListData = {}
    for trackTitle in cleanedTracks:
        time.sleep(3)
        # Search using---------------------- Youtube API
        #actualVideoId = searchUsingAPI(youtube,trackTitle)
        # Search Using---------------------- Youtube Crawler
        actualVideoId = YouTubeSearchEngine.searchUsingScraper(trackTitle)
        if actualVideoId == "Video Not Found":
            print(actualVideoId)
        elif actualVideoId == "Failed To Generate URL":
            print(actualVideoId)
        else:
            playListData[actualVideoId] = trackTitle
            insertPlaylistItem(playlistId,actualVideoId)
    return playListData

def insertPlaylistItem(playlistId,actualVideoId):
    insertVideo = youtube.playlistItems().insert(part='snippet',body={
      "snippet": {
        "playlistId": playlistId,
        "resourceId": {
          "kind": "youtube#video",
          "videoId": actualVideoId
        }
      }
    }).execute()

if __name__ == '__main__':
    flg = True
    while flg:
        tracklistCleanedName,cleanedTracks = TracklistGenerator.generateTracklist()
        service = int(input('Please select Service to save Tracks: 1.Youtube 2.Spotify [1 or 2] :'))
        if service == 1:
            youtube = OAuthVerification()
            playlistId = insertPlaylist(youtube,tracklistCleanedName,cleanedTracks)
            playListData = insertTracks(youtube,playlistId,cleanedTracks)
            prompt = str(input("Do You Want to Download the Tracks Locally ? [Yes/No]: ")).lower()
            if prompt == 'yes' or prompt == 'y' and playListData:
                getmp3(playListData)
                print("Tracks Downloaded")
            elif not playListData:
                print("PlayList is empty")
            else:
                print("Tracks Successfully Added on Youtube")
                
        elif service == 2:
            if SpotifyConnector(tracklistCleanedName,cleanedTracks): # Returns a Boolean
                print("Successfully created a Spotify Playlist")
            else:
                print("Failed to create a Spotify Playlist")
        rechoice = input('Do You Want To Add More Tracks Mate !? [Yes/No]')
        if rechoice.lower() == 'yes':
            flg = True
        else:
            flg = False

C:\Users\omkar.kadam\AppData\Local\Continuum\anaconda3\lib\site-packages\pydub\utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


Enter Tracklist URL : https://www.1001tracklists.com/tracklist/qkybr0k/suyano-revealed-radio-244-2019-11-22.html

Fetching Tracklist.....

Cleaning Tracklist.....

Tracks To Add : 19
NOTD X Daya - Wanted (RetroVision Remix)
SWACQ - Switch Back
Wildvibes & Patrick Key ft. Stine Kristiansen - Love Again
DubVision - Young Money
Gregor Salto - Azumba (Wiwek & Gregor Salto Remix)
Lewis Capaldi - Someone You Loved
Tom & Jame - New World Sound
Dimitri Vangelis & Wyman & Dzeko vs. Martin Garrix & BONN - The King vs. No Sleep (Fuerte & Whaler Mashup)
TV Noise - Mumble
Martin Garrix ft. Bonn - No Sleep
Teamworx & Thomas Feelman - Let It Sound
DubVision - Antares
Dimitri Vangelis & Wyman vs. Dzeko - The King
Daffy Muffin - Selecta
Julian Jordan - Bassline
Suyano - Playing In The Sun
DubVision vs. Lewis Capaldi - Antares vs. Someone You Loved (Suyano Mashup)
Suyano & Wasback ft. Daimy Lotus - Colors
Magnificence & 7 Skies - The Drill
Please select Service to save Tracks: 1.Youtube 2.Spotify [1 or 